# What if California gave money to everyone, not only vehicle owners?

Last month, Governor Gavin Newsom [proposed an $11 billion package](https://www.gov.ca.gov/2022/03/23/governor-newsom-proposes-11-billion-relief-package-for-californians-facing-higher-gas-prices/) for Californians facing higher gas prices. Newsom reserved $9 billion of this to send each vehicle owner $400 for each registered vehicle, up to two per person.

In this report, we estimate the poverty impact of this policy and compare it against budget-neutral alternatives that include Californians who don't own vehicles. We find that the vehicle subsidy will reduce the poverty rate by around a third as much as a universal basic income scheme with the same total cost.

In [1]:
from openfisca_us import Microsimulation, ACS
from ca_ubi.vehicle_subsidy import vehicle_subsidy
import numpy as np
from ca_ubi.ubi import create_ubi_reform

def set_non_ca_weights_to_zero(simulation: Microsimulation):
    person_weight = simulation.calc("person_weight").values
    simulation.set_input("person_weight", 2022, np.where(simulation.calc("state_code", map_to="person").values == "CA", person_weight, 0))
    household_weight = simulation.calc("household_weight").values
    simulation.set_input("household_weight", 2022, np.where(simulation.calc("state_code", map_to="household").values == "CA", household_weight, 0))
    return simulation

baseline = Microsimulation(dataset=ACS, year=2019)
baseline = set_non_ca_weights_to_zero(baseline)
vehicle_subsidy_simulation = Microsimulation(vehicle_subsidy, dataset=ACS, year=2019)
vehicle_subsidy_simulation = set_non_ca_weights_to_zero(vehicle_subsidy_simulation)
cost_of_vehicle_subsidy = vehicle_subsidy_simulation.calc("ca_vehicle_subsidy_payment", map_to="household").sum()
ca_population = baseline.calc("people").sum()
ubi_amount = cost_of_vehicle_subsidy / ca_population
print(f"Vehicle subsidy costs ${cost_of_vehicle_subsidy / 1e9:.1f}bn, CA population == {ca_population / 1e6:.1f} million, UBI amount == ${ubi_amount:.0f}/year")
ubi_simulation = Microsimulation(create_ubi_reform(ubi_amount), dataset=ACS, year=2019)
ubi_simulation = set_non_ca_weights_to_zero(ubi_simulation)

/home/nikhil/policyengine/openfisca-us/openfisca_us/variables/demographic/person/vehicles_owned.py:28: RuntimeWarning: invalid value encountered in remainder
  % num_adults_in_household


Vehicle subsidy costs $9.9bn, CA population == 38.7 million, UBI amount == $256/year


In [2]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected" # As per https://github.com/microsoft/vscode-jupyter/issues/4364#issuecomment-817352686
from ubicenter import format_fig
from ubicenter.plotly import BLUE_COLOR_SEQUENCE


def get_poverty_rates(simulation: Microsimulation, label: str, deep_poverty: bool = False) -> pd.DataFrame:
    in_poverty = simulation.calc(f"spm_unit_is_in{'_deep' if deep_poverty else ''}_spm_poverty", map_to="person")
    group_names = ["Children", "Adults", "Seniors", "All"]
    poverty_rates = []
    for group in ("is_child", "is_wa_adult", "is_senior", "people"):
        poverty_rates.append(in_poverty[simulation.calc(group) > 0].mean())
    return pd.DataFrame({
        "Group": group_names,
        "Poverty rate": poverty_rates,
        "Label": label,
        "Type": "Poverty" if not deep_poverty else "Deep poverty",
    })

def get_regular_and_deep_poverty_rates(simulation: Microsimulation, label: str) -> pd.DataFrame:
    regular_df = get_poverty_rates(simulation, label, deep_poverty=False)
    deep_df = get_poverty_rates(simulation, label, deep_poverty=True)
    return pd.concat([regular_df, deep_df])

def get_hovercards(poverty_rate_df: pd.DataFrame) -> list:
    hover_cards = []
    for i, row in poverty_rate_df.iterrows():
        poverty_label = "poverty" if row["Type"] == "Poverty" else "deep poverty"
        if row["Label"] == "Baseline":
            hover = f"The current {poverty_label} rate for <b>{row['Group'].lower()}</b> is <b>{row['Poverty rate']:.1%}</b>"
        else:
            baseline_rate = poverty_rate_df.loc[(poverty_rate_df["Label"] == "Baseline") & (poverty_rate_df["Type"] == row["Type"]), "Poverty rate"].iloc[0]
            if row["Label"] == "Vehicle subsidy":
                # Something like: "Under the vehicle subsidy, the poverty rate for children would rise/fall to x% (+/- y%)"
                hover = f"Under the vehicle subsidy, the{poverty_label} rate for <b>{row['Group'].lower()}</b> would fall<br> to <b>{row['Poverty rate']:.1%}</b> ({(row['Poverty rate'] - baseline_rate) / baseline_rate:.1%})"
            else:
                # Something like: "Under a UBI, the poverty rate for children would rise/fall to x% (+/- y%)"
                hover = f"Under a UBI, the {poverty_label} rate for <b>{row['Group'].lower()}</b> would fall<br> to <b>{row['Poverty rate']:.1%}</b> ({(row['Poverty rate'] - baseline_rate) / baseline_rate:.1%})"
        hover_cards.append(hover)
    return hover_cards

simulations = [baseline, vehicle_subsidy_simulation, ubi_simulation]
labels = ["Baseline", "Vehicle subsidy", "UBI"]
poverty_rate_df = pd.concat([get_regular_and_deep_poverty_rates(simulation, label) for simulation, label in zip(simulations, labels)])
poverty_rate_df["hover_card"] = get_hovercards(poverty_rate_df)

fig = px.bar(
    poverty_rate_df, 
    x="Group", 
    y="Poverty rate", 
    color="Label", 
    barmode="group", 
    color_discrete_sequence=BLUE_COLOR_SEQUENCE, 
    custom_data=["hover_card"],
    animation_frame="Type",
).update_layout(
    title="Poverty rate by age group",
    yaxis_tickformat=".0%",
    legend_title=""
).update_traces(
    hovertemplate="%{customdata[0]}",
)
format_fig(fig)

In [21]:
def get_decile_df(baseline: Microsimulation, simulation: Microsimulation, label: str) -> pd.DataFrame:
    income = baseline.calc("spm_unit_net_income", map_to="household")
    new_income = simulation.calc("spm_unit_net_income", map_to="household")
    if label == "Vehicle subsidy":
        new_income += simulation.calc("ca_vehicle_subsidy_payment", map_to="household")
    elif label == "UBI":
        new_income += simulation.calc("ca_basic_income", map_to="household")
    people = simulation.calc("people", map_to="household")
    income.weights *= people.values
    income_decile = income.decile_rank()
    income.weights /= people.values
    gain = new_income.groupby(income_decile).sum() / income.groupby(income_decile).sum() - 1
    gain = gain[1:]
    return pd.DataFrame({
        "Decile": gain.index,
        "Change to net income": gain.values,
        "Reform": label,
    })

reform_simulations = [vehicle_subsidy_simulation, ubi_simulation]
labels = ["Vehicle subsidy", "UBI"]
decile_df = pd.concat([get_decile_df(baseline, simulation, label) for simulation, label in zip(reform_simulations, labels)])

fig = px.bar(
    decile_df, 
    y="Change to net income", 
    x="Decile", 
    color="Reform", 
    barmode="group", 
    color_discrete_sequence=BLUE_COLOR_SEQUENCE
).update_layout(
    title="Change to net income by decile",
    yaxis_tickformat=".1%",
    xaxis_tickvals=list(range(1, 11)),
    legend_title=""
)
format_fig(fig)

In [36]:
df = pd.DataFrame({
    "State": baseline.calc("state_name"),
    "In poverty": baseline.calc("spm_unit_is_in_spm_poverty", map_to="household"),
    "In deep poverty": baseline.calc("spm_unit_is_in_deep_spm_poverty", map_to="household"),
    "Poverty threshold": baseline.calc("spm_unit_spm_threshold", map_to="household"),
    "Original net income": baseline.calc("spm_unit_net_income", map_to="household").round(),
    "Weight": baseline.calc("people", map_to="household").weights,
    "Adults": baseline.calc("is_adult", map_to="household"),
    "Children": baseline.calc("is_child", map_to="household"),
    "Vehicle subsidy": vehicle_subsidy_simulation.calc("ca_vehicle_subsidy_payment", map_to="household"),
    "UBI": ubi_simulation.calc("ca_basic_income", map_to="household"),
})

df[df.State == "CA"][df["In poverty"] > 0].sample(10)

/home/nikhil/anaconda3/envs/policyengine/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,State,In poverty,In deep poverty,Poverty threshold,Original net income,Weight,Adults,Children,Vehicle subsidy,UBI
153103,CA,1.0,1.0,45563.804688,14564.0,333.0,4.0,0.0,400.0,1024.376099
96234,CA,1.0,0.0,17331.443359,16767.0,91.0,1.0,0.0,0.0,256.094025
178326,CA,1.0,0.0,17382.304688,14344.0,29.0,1.0,0.0,400.0,256.094025
194271,CA,1.0,1.0,14288.376953,6500.0,43.0,1.0,0.0,0.0,256.094025
154435,CA,1.0,0.0,28439.611328,26357.0,14.0,2.0,2.0,800.0,1024.376099
73864,CA,4.0,3.0,63729.871094,15096.0,42.0,4.0,0.0,800.0,1024.376099
176149,CA,1.0,0.0,17126.876953,11190.0,55.0,1.0,0.0,0.0,256.094025
179645,CA,1.0,1.0,20146.611328,-3901.0,59.0,2.0,0.0,0.0,512.188049
134952,CA,1.0,0.0,24344.291016,15880.0,203.0,2.0,0.0,400.0,512.188049
144355,CA,1.0,0.0,11233.043945,9586.0,20.0,1.0,0.0,0.0,256.094025
